## 基于深度学习的文本分类

- Word2Vec
- 全连接神经网络
- LSTM
- GRU

In [18]:
import pandas as pd
import numpy as np

### 读取数据

In [12]:
data = pd.read_csv("all_data.csv")
data.head()

,分类,正文,文本分词
0,艺术,﻿【 文献号 】1-2432\r\n【原文出处】出版发行研究\r\n【原刊地名】京\r\n【...,﻿ 【 文献号 】 1 - 2432 \r\n 【 原文 出处 】 出版发行 研究 ...
1,艺术,﻿【 文献号 】1-2435\r\n【原文出处】扬州师院学报：社科版\r\n【原刊期号】19...,﻿ 【 文献号 】 1 - 2435 \r\n 【 原文 出处 】 扬州 师院 学报...
2,艺术,﻿【 文献号 】1-2785\r\n【原文出处】南通师专学报：社科版\r\n【原刊期号】19...,﻿ 【 文献号 】 1 - 2785 \r\n 【 原文 出处 】 南通 师专 学报...
3,艺术,﻿【 文献号 】1-3021\r\n【原文出处】社会科学战线\r\n【原刊地名】长春\r\n...,﻿ 【 文献号 】 1 - 3021 \r\n 【 原文 出处 】 社会科学 战线 ...
4,艺术,﻿【 文献号 】1-3062\r\n【原文出处】上海文化\r\n【原刊期号】199505\r...,﻿ 【 文献号 】 1 - 3062 \r\n 【 原文 出处 】 上海 文化 \r...


### 编码类别标签

In [13]:
from sklearn import preprocessing

encode = preprocessing.LabelEncoder()
label =  encode.fit_transform(data['分类'].values)

### 划分训练集和验证集

In [14]:
from sklearn.model_selection import train_test_split

X_train,X_valid,Y_train,Y_valid = train_test_split(data['文本分词'].values, label,stratify=label, random_state=42, test_size=0.1,shuffle=True ) 
# stratify=y : 按照数据集中y的比例分配给train和test，使得train和test中各类别数据的比例与原数据集的比例一致。
print (X_train.shape)
print (X_valid.shape)

(8324,)
(925,)


### 训练词向量

In [15]:
X=data['文本分词']
X = [i.split() for i in X]

In [5]:
import gensim

# 训练100维词向量
model = gensim.models.Word2Vec(X, min_count=5, window=8, size=100) # X 是分词后的文本构成的列表
embed_indx = dict(zip(model.wv.index2word, model.wv.vectors)) # 字典：词-词向量
print("词向量数量：%s" % len(embed_indx))

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


词向量数量：119775


### 文本向量化

In [19]:
import jieba
from tqdm import tqdm

stop_words = [line.strip() for line in open('中文停用词表.txt', 'r', encoding='utf-8').readlines()] # 停用词表

# 将语句转化为一个标准化的向量：各个词的向量求和再标准化
def sen2vec(s,model):

    words = [w for w in s.split() if w not in stop_words]
    M = []
    for w in words:
        try:
            M.append(model[w]) # 查询对应的词向量
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0) # 每一列求和
    if type(v) !=np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v**2).sum()) # 标准化

X_train_wv = [sen2vec(x,model) for x in tqdm(X_train)]
X_valid_wv = [sen2vec(x,model) for x in tqdm(X_valid)]

X_train_wv = np.array(X_train_wv)
X_valid_wv = np.array(X_valid_wv)



  0%|                                                                                         | 0/8324 [00:00<?, ?it/s]D:\Software_install_here\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


  0%|                                                                                 | 5/8324 [00:00<03:35, 38.63it/s]

  0%|                                                                                 | 7/8324 [00:00<04:39, 29.78it/s]

  0%|                                                                                 | 9/8324 [00:00<06:18, 22.00it/s]

  0%|                                                                                | 13/8324 [00:00<05:28, 25.30it/s]

  0%|▏                                                                               | 17/8324 [00:00<05:11, 26.66it/s]

  0%|▏                                              

  3%|██▌                                                                            | 272/8324 [00:08<03:57, 33.96it/s]

  3%|██▌                                                                            | 276/8324 [00:09<04:06, 32.68it/s]

  3%|██▋                                                                            | 280/8324 [00:09<04:44, 28.30it/s]

  3%|██▋                                                                            | 285/8324 [00:09<04:38, 28.85it/s]

  4%|██▊                                                                            | 292/8324 [00:09<03:54, 34.28it/s]

  4%|██▊                                                                            | 296/8324 [00:09<04:04, 32.87it/s]

  4%|██▊                                                                            | 302/8324 [00:09<03:50, 34.75it/s]

  4%|██▉                                                                            | 307/8324 [00:09<03:37, 36.92it/s]

  4%|██▉                        

  7%|█████▍                                                                         | 572/8324 [00:18<04:52, 26.51it/s]

  7%|█████▍                                                                         | 577/8324 [00:18<04:43, 27.36it/s]

  7%|█████▌                                                                         | 580/8324 [00:18<05:26, 23.70it/s]

  7%|█████▌                                                                         | 583/8324 [00:18<05:10, 24.96it/s]

  7%|█████▌                                                                         | 588/8324 [00:18<04:30, 28.61it/s]

  7%|█████▋                                                                         | 593/8324 [00:18<04:02, 31.87it/s]

  7%|█████▋                                                                         | 597/8324 [00:18<04:13, 30.53it/s]

  7%|█████▋                                                                         | 601/8324 [00:19<03:56, 32.69it/s]

  7%|█████▊                     

 11%|████████▍                                                                      | 889/8324 [00:27<03:40, 33.64it/s]

 11%|████████▍                                                                      | 893/8324 [00:27<03:47, 32.73it/s]

 11%|████████▌                                                                      | 899/8324 [00:27<03:23, 36.50it/s]

 11%|████████▌                                                                      | 903/8324 [00:27<03:29, 35.42it/s]

 11%|████████▌                                                                      | 907/8324 [00:27<04:01, 30.74it/s]

 11%|████████▋                                                                      | 912/8324 [00:27<03:36, 34.29it/s]

 11%|████████▋                                                                      | 916/8324 [00:27<03:57, 31.17it/s]

 11%|████████▋                                                                      | 920/8324 [00:28<03:53, 31.72it/s]

 11%|████████▊                  

 14%|███████████▏                                                                  | 1188/8324 [00:36<03:53, 30.54it/s]

 14%|███████████▏                                                                  | 1192/8324 [00:36<03:44, 31.73it/s]

 14%|███████████▏                                                                  | 1196/8324 [00:36<04:05, 29.06it/s]

 14%|███████████▏                                                                  | 1200/8324 [00:36<04:24, 26.90it/s]

 14%|███████████▎                                                                  | 1203/8324 [00:36<04:37, 25.65it/s]

 15%|███████████▎                                                                  | 1207/8324 [00:36<04:20, 27.36it/s]

 15%|███████████▎                                                                  | 1212/8324 [00:36<03:45, 31.49it/s]

 15%|███████████▍                                                                  | 1219/8324 [00:37<03:17, 35.99it/s]

 15%|███████████▍               

...

 96%|██████████████████████████████████████████████████████████████████████████▋   | 7973/8324 [04:20<00:13, 26.36it/s]

 96%|██████████████████████████████████████████████████████████████████████████▋   | 7976/8324 [04:21<00:13, 25.72it/s]

 96%|██████████████████████████████████████████████████████████████████████████▊   | 7979/8324 [04:21<00:13, 25.78it/s]

 96%|██████████████████████████████████████████████████████████████████████████▊   | 7982/8324 [04:21<00:13, 25.43it/s]

 96%|██████████████████████████████████████████████████████████████████████████▊   | 7987/8324 [04:21<00:12, 26.07it/s]

 96%|██████████████████████████████████████████████████████████████████████████▊   | 7990/8324 [04:21<00:13, 24.99it/s]

 96%|██████████████████████████████████████████████████████████████████████████▉   | 7995/8324 [04:21<00:11, 27.94it/s]

 96%|██████████████████████████████████████████████████████████████████████████▉   | 8002/8324 [04:21<00:09, 32.55it/s]

 96%|███████████████████████████

 99%|█████████████████████████████████████████████████████████████████████████████▏| 8233/8324 [04:30<00:03, 24.69it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▏| 8237/8324 [04:30<00:03, 27.04it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▏| 8240/8324 [04:30<00:03, 25.28it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▎| 8245/8324 [04:30<00:02, 29.66it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▎| 8250/8324 [04:31<00:02, 30.04it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▎| 8254/8324 [04:31<00:02, 28.72it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▍| 8258/8324 [04:31<00:02, 30.14it/s]

 99%|█████████████████████████████████████████████████████████████████████████████▍| 8262/8324 [04:31<00:02, 25.62it/s]

 99%|███████████████████████████

 21%|████████████████▌                                                               | 191/925 [00:05<00:31, 23.00it/s]

 21%|████████████████▊                                                               | 194/925 [00:06<00:30, 23.70it/s]

 22%|█████████████████▏                                                              | 199/925 [00:06<00:27, 26.09it/s]

 22%|█████████████████▋                                                              | 205/925 [00:06<00:23, 31.21it/s]

 23%|██████████████████                                                              | 209/925 [00:06<00:26, 27.12it/s]

 23%|██████████████████▍                                                             | 213/925 [00:06<00:25, 27.72it/s]

 23%|██████████████████▊                                                             | 217/925 [00:06<00:26, 26.58it/s]

 24%|███████████████████                                                             | 221/925 [00:06<00:24, 29.30it/s]

 24%|███████████████████▍       

 52%|█████████████████████████████████████████▌                                      | 480/925 [00:14<00:13, 33.92it/s]

 52%|█████████████████████████████████████████▊                                      | 484/925 [00:15<00:14, 30.36it/s]

 53%|██████████████████████████████████████████▏                                     | 488/925 [00:15<00:13, 32.39it/s]

 53%|██████████████████████████████████████████▌                                     | 492/925 [00:15<00:16, 26.85it/s]

 54%|██████████████████████████████████████████▊                                     | 495/925 [00:15<00:17, 24.16it/s]

 54%|███████████████████████████████████████████                                     | 498/925 [00:15<00:17, 24.40it/s]

 54%|███████████████████████████████████████████▌                                    | 504/925 [00:15<00:14, 29.04it/s]

 55%|███████████████████████████████████████████▉                                    | 508/925 [00:15<00:15, 26.57it/s]

 55%|███████████████████████████

 82%|█████████████████████████████████████████████████████████████████▌              | 758/925 [00:24<00:05, 29.46it/s]

 82%|█████████████████████████████████████████████████████████████████▉              | 762/925 [00:24<00:05, 28.04it/s]

 83%|██████████████████████████████████████████████████████████████████▏             | 766/925 [00:24<00:05, 27.47it/s]

 83%|██████████████████████████████████████████████████████████████████▌             | 769/925 [00:25<00:06, 24.01it/s]

 84%|██████████████████████████████████████████████████████████████████▊             | 773/925 [00:25<00:05, 26.98it/s]

 84%|███████████████████████████████████████████████████████████████████             | 776/925 [00:25<00:05, 27.11it/s]

 84%|███████████████████████████████████████████████████████████████████▍            | 780/925 [00:25<00:04, 29.63it/s]

 85%|███████████████████████████████████████████████████████████████████▉            | 785/925 [00:25<00:04, 32.70it/s]

 85%|███████████████████████████

### 多分类损失函数

In [23]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """
     对数损失度量的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1])) # 将真实标签转化为矩阵[0,0,1,0,0]
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps) # eps最小，1-eps最大，避免求对数出现问题
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip)) # 两个矩阵各元素乘机之和
    return -1.0 / rows * vsota # 负的交叉熵之和除以样本数

### Xgboost

In [26]:
import xgboost as xgb
from sklearn.metrics import classification_report

boost = xgb.XGBClassifier(max_depth=7,n_estimator=200, colsample_bytree=0.8,subsample=0.8,nthread=-1,learning_rate=0.1,silent=False)
boost.fit(X_train_wv, Y_train)
pred = boost.predict_proba(X_valid_wv)

print("多分类交叉熵损失：%0.3f" % multiclass_logloss(Y_valid,pred))
#print(classification_report(pred, Y_valid))

多分类交叉熵损失：0.390


## 深度学习算法

### 数据准备

In [28]:
# 数据标准化
from sklearn import preprocessing

scale = preprocessing.StandardScaler()
X_train_wv_st = scale.fit_transform(X_train_wv)
X_valid_wv_st = scale.transform(X_valid_wv)

# 编码类别
from keras.utils import np_utils

Y_train_en = np_utils.to_categorical(Y_train) # 将整型标签转为onehot
Y_valid_en = np_utils.to_categorical(Y_valid)

Using TensorFlow backend.


### 全连接神经网络

In [51]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.normalization import BatchNormalization

from keras import metrics

model = Sequential()
model.add(Dense(500,input_dim=100,activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300,activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(19))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[ metrics.categorical_accuracy]) 
# categorical_accuracy:对多分类问题,计算再所有预测值上的平均正确率

history = model.fit(X_train_wv_st,y=Y_train_en,batch_size=64,epochs=5,verbose=2,validation_data=(X_valid_wv_st,Y_valid_en))

Train on 8324 samples, validate on 925 samples
Epoch 1/5
 - 3s - loss: 0.8660 - categorical_accuracy: 0.7464 - val_loss: 0.4877 - val_categorical_accuracy: 0.8497
Epoch 2/5
 - 1s - loss: 0.5082 - categorical_accuracy: 0.8387 - val_loss: 0.4524 - val_categorical_accuracy: 0.8659
Epoch 3/5
 - 1s - loss: 0.4297 - categorical_accuracy: 0.8616 - val_loss: 0.3940 - val_categorical_accuracy: 0.8714
Epoch 4/5
 - 1s - loss: 0.3808 - categorical_accuracy: 0.8749 - val_loss: 0.3702 - val_categorical_accuracy: 0.8735
Epoch 5/5
 - 1s - loss: 0.3786 - categorical_accuracy: 0.8773 - val_loss: 0.3462 - val_categorical_accuracy: 0.8757


In [52]:
history_dict = history.history
history_dict.keys()

dict_keys(['val_loss', 'val_categorical_accuracy', 'loss', 'categorical_accuracy'])

### LSTM

#### 数据预处理

In [53]:
from keras.preprocessing import sequence, text

# https://blog.csdn.net/zzulp/article/details/76146947

token = text.Tokenizer(num_words=None) # 用来对文本中的词进行统计计数，生成文档词典，以支持基于词典位序生成文本的向量表示
max_len = 70

token.fit_on_texts(list(X_train) + list(X_valid)) #  使用一系列文档来生成token词典，texts为list类，每个元素为一个文档
X_train_seq = token.texts_to_sequences(X_train) #  将多个文档转换为word下标的向量形式,shape为[len(texts)，len(text)] 
X_valid_seq = token.texts_to_sequences(X_valid) # [[1, 2, 3, 4], [1, 2, 3, 5]] # 词的编号
# 通过词向量所处W矩阵的行号（index化）来表示原文，极大缩减了存储空间，还减少了训练时候的IO开销

X_train_pad = sequence.pad_sequences(X_train_seq,maxlen=max_len) #  将序列填充到maxlen长度,默认填0
X_valid_pad = sequence.pad_sequences(X_valid_seq,maxlen=max_len)
# pad_sequences(sequences, maxlen, padding=’pre’, truncating=’pre’, value=0.) 

word_index = token.word_index # 一个dict，保存所有word对应的编号id，从1开始
# 全部词向量组成的矩阵,增加0对应的词向量
embed_matrix = np.zeros((len(word_index)+1, 100)) ####### 加1的原因是上面补0了  http://blog.sina.com.cn/s/blog_1450ac3c60102x79x.html
for word , i in tqdm(word_index.items()): # 词，编号
    embed_vec = embed_indx.get(word) # 从词-词向量 字典查询对应的词向量
    if embed_vec is not None:
        embed_matrix[i] = embed_vec



  0%|                                                                                       | 0/381291 [00:00<?, ?it/s]

  1%|▉                                                                        | 4807/381291 [00:00<00:07, 47760.89it/s]

  3%|██                                                                      | 10837/381291 [00:00<00:07, 49667.89it/s]

  5%|███▎                                                                    | 17794/381291 [00:00<00:06, 54239.50it/s]

  7%|████▊                                                                   | 25342/381291 [00:00<00:06, 59138.76it/s]

  9%|██████▋                                                                 | 35421/381291 [00:00<00:05, 67444.67it/s]

 12%|████████▋                                                               | 46043/381291 [00:00<00:04, 73412.15it/s]

 15%|██████████▌                                                             | 56048/381291 [00:00<00:04, 79409.30it/s]

 17%|████████████▌            

In [54]:
embed_matrix.shape # 全部词向量矩阵

(381292, 100)

In [55]:
X_train_pad.shape # 每条文档补齐为一样长度：70

(8324, 70)

#### 构建模型

In [61]:
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import  SpatialDropout1D

from keras.callbacks import EarlyStopping

model = Sequential()

model.add(Embedding(len(word_index)+1,100,weights=[embed_matrix],input_length=max_len,trainable=False)) 
# Embedding层可理解为将词编号映射为词向量
# 词汇表大小，词向量的维度，(矩阵W的行数和列数),weights就是权重矩阵,input_length 输入序列的长度，trainable采用训练好的词向量，所以False

model.add(SpatialDropout1D(0.3))
model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(19))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[ metrics.categorical_accuracy]) 
# categorical_accuracy:对多分类问题,计算再所有预测值上的平均正确率
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

# X_train_pad 是词编号列表，
history = model.fit(X_train_pad,y=Y_train_en,batch_size=512,epochs=100,verbose=2,validation_data=(X_valid_pad,Y_valid_en),callbacks=[earlystop])

Train on 8324 samples, validate on 925 samples
Epoch 1/100
 - 15s - loss: 2.4188 - categorical_accuracy: 0.2071 - val_loss: 1.9807 - val_categorical_accuracy: 0.3092
Epoch 2/100
 - 21s - loss: 2.0502 - categorical_accuracy: 0.2841 - val_loss: 1.8419 - val_categorical_accuracy: 0.3546
Epoch 3/100
 - 21s - loss: 1.9339 - categorical_accuracy: 0.3203 - val_loss: 1.7412 - val_categorical_accuracy: 0.3805
Epoch 4/100
 - 18s - loss: 1.8530 - categorical_accuracy: 0.3555 - val_loss: 1.6788 - val_categorical_accuracy: 0.3989
Epoch 5/100
 - 16s - loss: 1.7976 - categorical_accuracy: 0.3800 - val_loss: 1.6098 - val_categorical_accuracy: 0.4378
Epoch 6/100
 - 16s - loss: 1.7456 - categorical_accuracy: 0.3969 - val_loss: 1.5563 - val_categorical_accuracy: 0.4541
Epoch 7/100
 - 22s - loss: 1.7012 - categorical_accuracy: 0.4163 - val_loss: 1.5149 - val_categorical_accuracy: 0.4746
Epoch 8/100
 - 16s - loss: 1.6572 - categorical_accuracy: 0.4296 - val_loss: 1.4707 - val_categorical_accuracy: 0.4876
E

## 双向LSTM

In [ ]:
from keras.layers import Bidirectional

model = Sequential()

model.add(Embedding(len(word_index)+1,100,weights=[embed_matrix],input_length=max_len,trainable=False)) 
# Embedding层可理解为将词编号映射为词向量
# 词汇表大小，词向量的维度，(矩阵W的行数和列数),weights就是权重矩阵,input_length 输入序列的长度，trainable采用训练好的词向量，所以False

model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(100,dropout=0.3,recurrent_dropout=0.3))) #### 双向LSTM区别

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(19))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[ metrics.categorical_accuracy]) 
# categorical_accuracy:对多分类问题,计算再所有预测值上的平均正确率
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

# X_train_pad 是词编号列表，
history = model.fit(X_train_pad,y=Y_train_en,batch_size=512,epochs=100,verbose=2,validation_data=(X_valid_pad,Y_valid_en),callbacks=[earlystop])

## GRU

In [ ]:
from keras.layers.recurrent import GRU

model = Sequential()

model.add(Embedding(len(word_index)+1,100,weights=[embed_matrix],input_length=max_len,trainable=False)) 
model.add(SpatialDropout1D(0.3))

# 两层GRU
model.add(GRU(100, dropout=0.3,recurrent_dropout=0.3,return_sequences=True))
model.add(GRU(100, dropout=0.3,recurrent_dropout=0.3))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(19))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=[ metrics.categorical_accuracy]) 

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
# X_train_pad 是词编号列表，
history = model.fit(X_train_pad,y=Y_train_en,batch_size=512,epochs=100,verbose=2,validation_data=(X_valid_pad,Y_valid_en),callbacks=[earlystop])